# Optimization - Exercise

Take the SMA LRM process and define an optimization problem to get the highest possible recovery with > 95% purity. The limitations are:
- 10 seconds of loading time + 80 seconds of wash, then gradients of any shape
- 6000 seconds maximal process duration.
- 1000 mM maximal salt concentration

You will have one night with 12 cores per group to run the optimization.